# **Test communication with database** -In this notebook we going to examine the dataengine , ingest and dataset with spark and creating the error detection part

For first step we create import the sufficient packages 

In [1]:
import sys
import os
sys.path.append('../')
from holoclean import dataengine , dataset , holocleansession

##  Specify the MySQL driver
For first step we need to add the connector for reading from mysql to JDBC and reverse 

In [2]:
my_path = os.path.abspath(os.path.dirname('__file__'))
path = os.path.join(my_path, "../holoclean/mysql-connector-java-5.1.44-bin.jar")
holoclean_se=holocleansession.HolocleanSession(path)

##  Creating Dataset and dataengine and holocleansession 
In this part we create dataset for keep tracking the runnig step and dataengine for make connector to the database we try to ingest the 10 element csv file and after ingesting it returns df other wise we can read it from database

In [3]:
ds=dataset.Dataset()  
d=dataengine.Dataengine("metadb-config.txt",'datadb-config.txt',ds,holoclean_se.return_sqlcontext())
holoclean_se.set_dataengine(d)
df=d.ingest_spark('10.csv',holoclean_se.returnspark_session())
df.show()

index
+-----+-----------+--------+--------+----+
|index|       city|    date|tempType|temp|
+-----+-----------+--------+--------+----+
|    1|ITE00100554|18000101|    TMAX| -75|
|    2|ITE00100554|18000101|    TMIN|-148|
|    3|GM000010962|18000101|    PRCP|   0|
|    4|EZE00100082|18000101|    TMAX| -86|
|    5|EZE00100082|18000101|    TMAX|-135|
|    6|ITE00100554|18000102|    TMAX| -60|
|    7|ITE00100554|18000102|    TMIN|-125|
|    8|GM000010962|18000102|    PRCP|   0|
|    9|EZE00100082|18000102|    TMAX| -44|
|   10|EZE00100082|18000102|    TMIN|-135|
+-----+-----------+--------+--------+----+



##  Read table from database
We can read table from database too

In [4]:
df=d.get_table_spark("T")
df.show()

+-----+-----------+--------+--------+----+
|index|       city|    date|tempType|temp|
+-----+-----------+--------+--------+----+
|    1|ITE00100554|18000101|    TMAX| -75|
|    2|ITE00100554|18000101|    TMIN|-148|
|    3|GM000010962|18000101|    PRCP|   0|
|    4|EZE00100082|18000101|    TMAX| -86|
|    5|EZE00100082|18000101|    TMAX|-135|
|    6|ITE00100554|18000102|    TMAX| -60|
|    7|ITE00100554|18000102|    TMIN|-125|
|    8|GM000010962|18000102|    PRCP|   0|
|    9|EZE00100082|18000102|    TMAX| -44|
|   10|EZE00100082|18000102|    TMIN|-135|
+-----+-----------+--------+--------+----+



In [5]:
dcCode=['t1&t2&EQ(t1.city,t2.city)&EQ(t1.temp,t2.temp)&IQ(t1.tempType,t2.tempType)']

dk_cells,clean_cells=holoclean_se._error_detection(dcCode)
dk_cells.show()

clean_cells.show()

ind
ind
+---+--------+
|ind|    attr|
+---+--------+
| 10|    temp|
|  5|tempType|
|  5|    city|
| 10|    city|
| 10|tempType|
|  5|    temp|
+---+--------+

+---+--------+
|ind|    attr|
+---+--------+
|  7|    city|
|  9|   index|
|  1|    temp|
|  5|    date|
|  7|    temp|
|  1|   index|
|  2|    city|
|  3|    city|
|  8|   index|
|  2|   index|
|  4|tempType|
|  2|    date|
|  2|    temp|
|  3|    date|
|  5|   index|
|  8|    date|
|  8|tempType|
|  8|    city|
|  9|    date|
| 10|   index|
+---+--------+
only showing top 20 rows



##  Error detection
This step create table of index and attribute which violate or make no violation by given denial constraints. In here we can get from data base or get it directly from error detection method

##  Call the table from database 
Here we call the table from database to spark dataframe too

In [6]:
dk_cells_in_db=d.get_table_spark("C_dk")

dk_cells_in_db.show(dk_cells_in_db.count())

+---+--------+
|ind|    attr|
+---+--------+
| 10|    temp|
|  5|    city|
|  5|tempType|
| 10|    city|
| 10|tempType|
|  5|    temp|
+---+--------+



We make a feature vector from denial constraints signals for all the cells in the dataset

In [7]:
holoclean_se._featurizer()

+------------+---+---+---+---+---+---+---+---+---+---+
|        cell|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10|
+------------+---+---+---+---+---+---+---+---+---+---+
|    [1,city]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [1,date]|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|[1,tempType]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [1,temp]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [2,city]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [2,date]|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|[2,tempType]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [2,temp]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [3,city]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [3,date]|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|[3,tempType]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [3,temp]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [4,city]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [4,date]|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|[4,tempType]|  1|  1|  1|  1|  1|  1|  1|  1|  1|  1|
|    [4,te